In [ ]:
import pkg_resources as pkg

path = pkg.resource_filename(__name__, "../")

import os

os.chdir(path)

In [2]:
import torch
from pytorch_lightning import LightningModule

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns
import pandas as pd

#from physioex.data import SleepPhysionet, TimeDistributedModule
#from physioex.data import datamodule
#from physioex.train.networks import TinySleepNet, ContrTinySleepNet
#from physioex.train.networks import seqsleepnet

In [3]:
def compute_projections(model, datamodule, contr=False):
    train_projections = []
    y_train_true = []
    y_train_pred = []
    first = True
    for batch in datamodule.train_dataloader():
        inputs, y_true = batch

        y_train_true.append(y_true)
        if contr:
            projections, y_pred = model(inputs.to(model.device))
        else:
            projections, y_pred = model.encode(inputs.to(model.device))

        y_train_pred.append(y_pred.cpu().detach().numpy())
        train_projections.append(projections.cpu().detach().numpy())

        del projections, y_pred

    y_train_true = np.concatenate(y_train_true).reshape(-1)
    train_projections = np.concatenate(train_projections).reshape(
        y_train_true.shape[0], -1
    )
    y_train_pred = np.concatenate(y_train_pred).reshape(-1, 5)

    return train_projections, y_train_true, y_train_pred

In [ ]:
dataset = SleepPhysionet(version="2018", use_cache=True)
dataset.split(fold=0)
sequence_length = 3
batch_size = 32
datamodule = TimeDistributedModule(
    dataset=dataset,
    sequence_lenght=sequence_length,
    batch_size=batch_size,
    transform=None,
    target_transform=None,
)

ccl_model_path = "models/CCL/tinysleepnet/seqlen=3/SleepPhysionet/2018/fold=0-epoch=15-step=77670-val_acc=0.79.ckpt"
scl_model_path = "models/SCL/tinysleepnet/seqlen=3/SleepPhysionet/2018/fold=0-epoch=18-step=93864-val_acc=0.79.ckpt"

In [ ]:
from physioex.models import load_pretrained_model
path_ssn = "/home/tinghi/physioex/models/ssn_scl/fold=-1-epoch=17-step=28348-val_acc=0.83.ckpt"
path_ssn_sum = "/home/tinghi/physioex/models/ssn_epseq_sum_scl/fold=-1-epoch=18-step=29323-val_acc=0.82.ckpt"
path_ssn_conc = "/home/tinghi/physioex/models/ssn_epseq_conc_scl/fold=-1-epoch=16-step=25482-val_acc=0.82.ckpt"
path_ssn_epoch = "/home/tinghi/physioex/models/ssn_ep_scl/fold=-1-epoch=17-step=28348-val_acc=0.81.ckpt"

seqsleepnet_model = load_pretrained_model("seqsleepnet", in_channels=1, sequence_length=10, loss="scl", ckpt_path=path_ssn).eval()
seqsleepnet_sum_model = load_pretrained_model("seqsleepnet", in_channels=1, sequence_length=10, loss="scl", ckpt_path=path_ssn_sum).eval()
seqsleepnet_conc_model = load_pretrained_model("seqsleepnet", in_channels=1, sequence_length=10, loss="scl", ckpt_path=path_ssn_conc).eval()
seqsleepnet_epoch_model = load_pretrained_model("seqsleepnet", in_channels=1, sequence_length=10, loss="scl", ckpt_path=path_ssn_epoch).eval()


In [19]:
from physioex.data.datamodule import PhysioExDataModule

datamodule = PhysioExDataModule(
    datasets=["mass"],
    versions=None,
    batch_size=15,
    selected_channels=["EEG"],
    sequence_length=10,
    data_folder="/mnt/guido-data/",
    preprocessing = "xsleepnet",
    target_transform= None,
)
print(len(datamodule.dataset))
print(len(datamodule.dataset[0]))
print(len(datamodule.dataset[0][0]))
print(datamodule.dataset[0][0][1].shape)
print(datamodule.dataset[2359][1])

2024-09-26 08:01:26.484 | INFO     | physioex.utils.data_folder:set_data_folder:25 - Data folder set to /mnt/guido-data/


227070
2
10
torch.Size([1, 29, 129])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


In [ ]:
import matplotlib.pyplot as plt
import torch

# Assuming the tensor is obtained from the dataset
tensor1 = datamodule.dataset.__getitem__(0)[0][0]
tensor2 = datamodule.dataset.__getitem__(1300)[0][0]

first_dim = torch.arange(29)
second_dim = torch.arange(129)
third_dim1 = tensor1[0, first_dim[:, None], second_dim]
third_dim2 = tensor2[0, first_dim[:, None], second_dim]
# Create a meshgrid for the x and y coordinates
X, Y = torch.meshgrid(first_dim, second_dim, indexing='ij')

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Plot the first spectrogram
c1 = ax1.pcolormesh(X, Y, third_dim1, shading='auto')
fig.colorbar(c1, ax=ax1, label='Intensity')
ax1.set_xlabel('First Dimension (x-coordinates)')
ax1.set_ylabel('Second Dimension (y-coordinates)')
ax1.set_title('Spectrogram 1')

# Plot the second spectrogram (assuming you have another tensor for the second plot)
# For demonstration, we'll use the same tensor
c2 = ax2.pcolormesh(X, Y, third_dim2, shading='auto')
fig.colorbar(c2, ax=ax2, label='Intensity')
ax2.set_xlabel('First Dimension (x-coordinates)')
ax2.set_ylabel('Second Dimension (y-coordinates)')
ax2.set_title('Spectrogram 2')


In [ ]:
# Get the train dataloader
train_dataloader = datamodule.train_dataloader()

# Create an iterator from the dataloader
train_dataloader_iter = iter(train_dataloader)

# Get the first element
first_element = next(train_dataloader_iter)

# Print the first element
print(first_element)

In [ ]:
# Get the train dataloader
train_dataloader = datamodule.train_dataloader()

# Create an iterator from the dataloader
train_dataloader_iter = iter(train_dataloader)

# Get the first element
first_element = next(train_dataloader_iter)

# Print the first element
print(first_element)

## PCA components plot

### Extracting the latent space projections from the similarity model

In [ ]:
seqsleepnet_model
seqsleepnet_sum_model
seqsleepnet_conc_model
seqsleepnet_epoch_model

In [ ]:
train_projections_ssn, y_train_true_ssn, y_train_pred_ssn = compute_projections(
    seqsleepnet_model, datamodule, contr=False
)
preds_snn = np.argmax(y_train_pred_ssn, axis=1)
print(train_projections_ssn.shape)

In [6]:
train_projections_ssn, y_train_true_ssn, y_train_pred_ssn = compute_projections(
    seqsleepnet_model, datamodule, contr=False
)
preds_snn = np.argmax(y_train_pred_ssn, axis=1)
print("ssn end")

train_projections_ssn_sum, y_train_true_ssn_sum, y_train_pred_ssn_sum = compute_projections(
    seqsleepnet_sum_model, datamodule, contr=False
)
preds_snn_sum = np.argmax(y_train_pred_ssn_sum, axis=1)
print("ssn sum end")

train_projections_ssn_conc, y_train_true_ssn_conc, y_train_pred_ssn_conc = compute_projections(
    seqsleepnet_conc_model, datamodule, contr=False
)
preds_snn_conc = np.argmax(y_train_pred_ssn_conc, axis=1)
print("ssn conc end")

train_projections_ssn_epoch, y_train_true_ssn_epoch, y_train_pred_ssn_epoch = compute_projections(
    seqsleepnet_epoch_model, datamodule, contr=False
)
preds_snn_epoch = np.argmax(y_train_pred_ssn_epoch, axis=1)
print("ssn epoch end")

ssn end
ssn sum end
ssn conc end
ssn epoch end


In [6]:
model = ContrTinySleepNet.load_from_checkpoint(scl_model_path).eval()
contr_train_projections, contr_y_train_true, contr_y_train_pred = compute_projections(
    model, datamodule, contr=True
)

model = TinySleepNet.load_from_checkpoint(ccl_model_path).eval()
train_projection, y_train_true, y_train_pred = compute_projections(
    model, datamodule, contr=False
)

# filtering
contr_preds = np.argmax(contr_y_train_pred, axis=1)
# contr_true_indx = np.where(contr_y_train_true == contr_preds or contr_y_train_true != contr_preds)[0]

preds = np.argmax(y_train_pred, axis=1)
# true_indx = np.where(y_train_true == preds or y_train_true != preds)[0]

In [7]:
true_indx_ssn = [True for i in range(len(y_train_true_ssn))]
true_indx_ssn_sum = [True for i in range(len(y_train_true_ssn_sum))]
true_indx_ssn_conc = [True for i in range(len(y_train_true_ssn_conc))]
true_indx_ssn_epoch = [True for i in range(len(y_train_true_ssn_epoch))]
#true_indx = [True for i in range(len(contr_y_train_true))]

### K-medoids

In [ ]:
import pandas as pd

# Assuming train_projections_ssn and y_train_true_ssn are lists or arrays
# Convert them to DataFrames if they are not already
train_projections_df = pd.DataFrame(train_projections_ssn)
y_train_true_df = pd.DataFrame(y_train_true_ssn, columns=['y_train_true_ssn'])

# Concatenate along the columns (axis=1)
result_df = pd.concat([train_projections_df, y_train_true_df], axis=1)
print(result_df.shape)
'''
print(result_df[result_df['y_train_true_ssn'] == 0].shape)
print(result_df[result_df['y_train_true_ssn'] == 1].shape)
print(result_df[result_df['y_train_true_ssn'] == 2].shape)
print(result_df[result_df['y_train_true_ssn'] == 3].shape)
print(result_df[result_df['y_train_true_ssn'] == 4].shape)'''

from sklearn_extra.cluster import KMedoids
# Sample the dataset
sampled_df = result_df.sample(n=10000, random_state=0)  # Adjust the sample size as needed

# Apply k-medoids clustering with k=1
kmedoids = KMedoids(n_clusters=1, random_state=0)
kmedoids.fit(sampled_df.iloc[:, :-1])

print(kmedoids.cluster_centers_)

### Plotting the first 2 PCA components

In [8]:
# Increase the number of components to see the variance in other components
pca = PCA(n_components=2)  # Adjust the number of components as needed

# First PCA transformation
principal_components_ssn = pca.fit_transform(train_projections_ssn[true_indx_ssn, :])
true_positive_ssn = y_train_true_ssn[true_indx_ssn]
explained_variance_ssn = pca.explained_variance_ratio_

df_ssn = pd.DataFrame(
    {
        "First Principal Component": principal_components_ssn[:, 0],
        "Second Principal Component": principal_components_ssn[:, 1],
        #"Third Principal Component": principal_components_ssn[:, 2],
        "Group": true_positive_ssn,
    }
)
print("Explained variance ratio for ssn:", explained_variance_ssn)
print("Explained variance:", explained_variance_ssn.sum())

# Second PCA transformation
principal_components_ssn_sum = pca.fit_transform(train_projections_ssn_sum[true_indx_ssn_sum, :])
true_positive_ssn_sum = y_train_true_ssn_sum[true_indx_ssn_sum]
explained_variance_ssn_sum = pca.explained_variance_ratio_

df_ssn_sum = pd.DataFrame(
    {
        "First Principal Component": principal_components_ssn_sum[:, 0],
        "Second Principal Component": principal_components_ssn_sum[:, 1],
        #"Third Principal Component": principal_components_ssn_sum[:, 2],
        "Group": true_positive_ssn_sum,
    }
)
print("Explained variance ratio for ssn_sum:", explained_variance_ssn_sum)
print("Explained variance:", explained_variance_ssn_sum.sum())

# Third PCA transformation
principal_components_ssn_conc = pca.fit_transform(train_projections_ssn_conc[true_indx_ssn_conc, :])
true_positive_ssn_conc = y_train_true_ssn_conc[true_indx_ssn_conc]
explained_variance_ssn_conc = pca.explained_variance_ratio_

df_ssn_conc = pd.DataFrame(
    {
        "First Principal Component": principal_components_ssn_conc[:, 0],
        "Second Principal Component": principal_components_ssn_conc[:, 1],
        #"Third Principal Component": principal_components_ssn_conc[:, 2],
        "Group": true_positive_ssn_conc,
    }
)
print("Explained variance ratio for ssn_conc:", explained_variance_ssn_conc)
print("Explained variance:", explained_variance_ssn_conc.sum())

# Fourth PCA transformation
principal_components_ssn_epoch = pca.fit_transform(train_projections_ssn_epoch[true_indx_ssn_epoch, :])
true_positive_ssn_epoch = y_train_true_ssn_epoch[true_indx_ssn_epoch]
explained_variance_ssn_epoch = pca.explained_variance_ratio_

df_ssn_epoch = pd.DataFrame(
    {
        "First Principal Component": principal_components_ssn_epoch[:, 0],
        "Second Principal Component": principal_components_ssn_epoch[:, 1],
        #"Third Principal Component": principal_components_ssn_epoch[:, 2],
        "Group": true_positive_ssn_epoch,
    }
)
print("Explained variance ratio for ssn_epoch:", explained_variance_ssn_epoch)
print("Explained variance:", explained_variance_ssn_epoch.sum())

Explained variance ratio for ssn: [0.3405441  0.17688899]
Explained variance: 0.5174331
Explained variance ratio for ssn_sum: [0.30972674 0.1700678 ]
Explained variance: 0.47979456
Explained variance ratio for ssn_conc: [0.27933824 0.1812165 ]
Explained variance: 0.46055472
Explained variance ratio for ssn_epoch: [0.1714484  0.08993109]
Explained variance: 0.26137948


In [9]:
classes = {0:"Wake", 1:"N1", 2:"N2", 3:"N3", 4:"REM"}
df_ssn["Group"] = df_ssn["Group"].map(classes)
df_ssn_epoch["Group"] = df_ssn_epoch["Group"].map(classes)
df_ssn_conc["Group"] = df_ssn_conc["Group"].map(classes)
df_ssn_sum["Group"] = df_ssn_sum["Group"].map(classes)

In [10]:
print("Base", df_ssn["Group"].unique())
print("Epoch", df_ssn_epoch["Group"].unique())
print("Conc", df_ssn_conc["Group"].unique())
print("Sum", df_ssn_sum["Group"].unique())

Base ['N2' 'N1' 'REM' 'N3' 'Wake']
Epoch ['Wake' 'N1' 'N2' 'N3' 'REM']
Conc ['REM' 'N3' 'N2' 'N1' 'Wake']
Sum ['N2' 'REM' 'N1' 'N3' 'Wake']


---

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 11))
classes=["Wake", "N1", "N2", "N3", "REM"]

sns.scatterplot(
    data=df_ssn_epoch[(df_ssn_epoch["First Principal Component"] >= -0.4)&(df_ssn_epoch["First Principal Component"] <= 0.4)],
    x="First Principal Component",
    y="Second Principal Component",
    hue="Group",
    hue_order=classes,
    palette="Set1",
    ax=axes[0,1],
).set(title="SeqSleepNet SCL Epoch encoding")

In [101]:
# Enable interactive plotting
%matplotlib notebook


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Define the classes variable
classes = ["Wake", "N1", "N2", "N3", "REM"]

fig = plt.figure(figsize=(8, 6))

# First subplot
ax1 = fig.add_subplot(1, 1, 1, projection='3d')
ax1.scatter(
    df_ssn["First Principal Component"],
    df_ssn["Second Principal Component"],
    df_ssn["Third Principal Component"],
    c=df_ssn["Group"].apply(lambda x: classes.index(x)),
    cmap="Set1"
)
ax1.set_title("SeqSleepNet SCL")
ax1.set_xlabel("First Principal Component")
ax1.set_ylabel("Second Principal Component")
ax1.set_zlabel("Third Principal Component")

plt.show()
plt.close()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Define the classes variable
classes = ["Wake", "N1", "N2", "N3", "REM"]

fig = plt.figure(figsize=(16, 11))

# First subplot
ax1 = fig.add_subplot(2, 2, 1, projection='3d')
ax1.scatter(
    df_ssn["First Principal Component"],
    df_ssn["Second Principal Component"],
    df_ssn["Third Principal Component"],
    c=df_ssn["Group"].apply(lambda x: classes.index(x)),
    cmap="Set1"
)
ax1.set_title("SeqSleepNet SCL")
ax1.set_xlabel("First Principal Component")
ax1.set_ylabel("Second Principal Component")
ax1.set_zlabel("Third Principal Component")

# Second subplot
ax2 = fig.add_subplot(2, 2, 2, projection='3d')
filtered_df = df_ssn_epoch[(df_ssn_epoch["First Principal Component"] >= -0.4) & (df_ssn_epoch["First Principal Component"] <= 0.4)]
ax2.scatter(
    filtered_df["First Principal Component"],
    filtered_df["Second Principal Component"],
    filtered_df["Third Principal Component"],
    c=filtered_df["Group"].apply(lambda x: classes.index(x)),
    cmap="Set1"
)
ax2.set_title("SeqSleepNet SCL Epoch encoding")
ax2.set_xlabel("First Principal Component")
ax2.set_ylabel("Second Principal Component")
ax2.set_zlabel("Third Principal Component")

# Third subplot
ax3 = fig.add_subplot(2, 2, 3, projection='3d')
ax3.scatter(
    df_ssn_sum["First Principal Component"],
    df_ssn_sum["Second Principal Component"],
    df_ssn_sum["Third Principal Component"],
    c=df_ssn_sum["Group"].apply(lambda x: classes.index(x)),
    cmap="Set1"
)
ax3.set_title("SeqsleepNet SCL Sum Epoch-Sequence")
ax3.set_xlabel("First Principal Component")
ax3.set_ylabel("Second Principal Component")
ax3.set_zlabel("Third Principal Component")

# Fourth subplot
ax4 = fig.add_subplot(2, 2, 4, projection='3d')
ax4.scatter(
    df_ssn_conc["First Principal Component"],
    df_ssn_conc["Second Principal Component"],
    df_ssn_conc["Third Principal Component"],
    c=df_ssn_conc["Group"].apply(lambda x: classes.index(x)),
    cmap="Set1"
)
ax4.set_title("SeqsleepNet SCL Concatenation Epoch-Sequence")
ax4.set_xlabel("First Principal Component")
ax4.set_ylabel("Second Principal Component")
ax4.set_zlabel("Third Principal Component")

plt.savefig("scatterplot_3d.png")
plt.show()
plt.close()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 11))
classes=["Wake", "N1", "N2", "N3", "REM"]
# Utilizza seaborn.scatterplot
sns.scatterplot(
    data=df_ssn,
    x="First Principal Component",
    y="Second Principal Component",
    hue="Group",
    hue_order=classes,
    palette="Set1",
    ax=axes[0, 0],
).set(title="SeqSleepNet SCL")

sns.scatterplot(
    data=df_ssn_epoch[(df_ssn_epoch["First Principal Component"] >= -0.4)&(df_ssn_epoch["First Principal Component"] <= 0.4)],
    x="First Principal Component",
    y="Second Principal Component",
    hue="Group",
    hue_order=classes,
    palette="Set1",
    ax=axes[0,1],
).set(title="SeqSleepNet SCL Epoch encoding")

sns.scatterplot(
    data=df_ssn_sum,
    x="First Principal Component",
    y="Second Principal Component",
    hue="Group",
    hue_order=classes,
    palette="Set1",
    ax=axes[1, 0],
).set(title="SeqsleepNet SCL Sum Epoch-Sequence")

sns.scatterplot(
    data=df_ssn_conc,
    x="First Principal Component",
    y="Second Principal Component",
    hue="Group",
    hue_order=classes,
    palette="Set1",
    ax=axes[1, 1],
).set(title="SeqsleepNet SCL Concatenation Epoch-Sequence")

plt.savefig("scatterplot.png")
plt.show()
plt.close()

### Assessing the latent space quality measuring the ARI

In [ ]:
K_values = range(2, 10)

# Lista per memorizzare i risultati ARI
ari_values = []
contr_ari_values = []
for K in K_values:
    # Esegui K-Means con il valore corrente di K
    kmeans = KMeans(n_clusters=K, random_state=0).fit(train_projection[true_indx, :])
    # Calcola l'ARI confrontando le assegnazioni di K-Means con i true_positive
    ari = adjusted_rand_score(true_positive, kmeans.labels_)
    # Memorizza il risultato
    ari_values.append(ari)

    # Esegui K-Means con il valore corrente di K
    kmeans = KMeans(n_clusters=K, random_state=0).fit(
        contr_train_projections[contr_true_indx, :]
    )
    # Calcola l'ARI confrontando le assegnazioni di K-Means con i true_positive
    ari = adjusted_rand_score(contr_true_positive, kmeans.labels_)
    # Memorizza il risultato
    contr_ari_values.append(ari)

In [ ]:
results = pd.DataFrame(
    {
        "Number of clusters (K)": K_values,
        "TinySleepNet": ari_values,
        "ContrTinySleepNet": contr_ari_values,
    }
)

# Plotta i risultati con Seaborn
plt.figure(figsize=(8, 6))
sns.lineplot(data=results[["TinySleepNet", "ContrTinySleepNet"]], marker="o").set(
    title="Adjusted Rand Index"
)
plt.show()
plt.close()

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(train_projection[true_indx, :])
contr_kmeans = KMeans(n_clusters=5, random_state=0).fit(
    contr_train_projections[contr_true_indx, :]
)

df["KMeans"] = kmeans.labels_
contr_df["KMeans"] = contr_kmeans.labels_

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 9))

# Utilizza seaborn.scatterplot
sns.scatterplot(
    data=df,
    x="First Principal Component",
    y="Second Principal Component",
    hue="KMeans",
    palette="Set1",
    ax=axes[0],
).set(title="TinySleepNet")
sns.scatterplot(
    data=contr_df,
    x="First Principal Component",
    y="Second Principal Component",
    hue="KMeans",
    palette="Set1",
    ax=axes[1],
).set(title="ContrTinySleepNet")

plt.show()
plt.close()

In [ ]:
import pkg_resources as pkg

path = pkg.resource_filename(__name__, "../")

import os

os.chdir(path)

In [ ]:
from physioex.explain.ari_explainer import ARIExplainer

expl = ARIExplainer(
    model_name="tinysleepnet",
    dataset_name="sleep_physionet",
    ckp_path="models/CCL/tinysleepnet/seqlen=3/SleepPhysionet/2018/",
    version="2018",
    use_cache=True,
    sequence_lenght=3,
    batch_size=32,
    n_jobs=10,
)

In [ ]:
expl.explain(plot_pca=True, plot_kmeans=True, n_jobs=1)